In [2]:
import pandas as pd
import altair as alt
import os
import pprint
from vega_datasets import data
import pycountry

C:\Users\kamin\AppData\Local\Temp\ipykernel_24004\488820849.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Create dataframes:
coastline_df = pd.read_csv('./data/coastline_lengths.csv', header=1, thousands=',')
ghg_df = pd.read_csv('./data/ghg_EDGAR_country.csv')
co2_df = pd.read_csv('./data/mean_co2_ppm.csv')
sea_level_df = pd.read_csv('./data/mean_sea_level_global.csv')
# coastline_df.head()
# co2_df.head()
co2_df.head()

,year,month,decimal_year,monthly_average,fld4,fld5,fld6,std_dev,fld8
0,1958,3,1958.2027,315.70,314.43,-1,-9.99,-0.9,NaN
1,1958,4,1958.2877,317.45,315.16,-1,-9.99,-0.9,NaN
2,1958,5,1958.3699,317.51,314.71,-1,-9.99,-0.9,NaN
3,1958,6,1958.4548,317.24,315.14,-1,-9.99,-0.9,NaN
4,1958,7,1958.5370,315.86,315.18,-1,-9.99,-0.9,NaN


In [10]:
# zooming and panning
zoom = alt.selection_interval(bind='scales', encodings=['x'])
nearest = alt.selection_point(on='mouseover', nearest=True, empty=False, encodings=['x'])

co2_chart = alt.Chart(co2_df).mark_line().encode(
    y = alt.Y('monthly_average:Q', title='CO2 (ppm)'),
    x = alt.X('decimal_year:Q', title='Year', axis=alt.Axis(format='d', title=None), scale=alt.Scale(domain=[1961, 2023])),
    tooltip=['decimal_year', 'monthly_average']
).add_params(
    zoom
)

# Vertical line
vertical_line = alt.Chart(co2_df).mark_rule(size=4, color='lightgray').encode(
    x='decimal_year:Q',
    opacity=alt.condition(nearest, alt.value(0.7), alt.value(0)),
).add_params(
    nearest
)

interaction_dots = co2_chart.mark_point(size=90, color='firebrick').transform_filter(
    nearest
).encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text_labels = co2_chart.mark_text(align='left', dx=-40, dy=-15).encode(
    text=alt.condition(nearest, alt.Text('monthly_average:Q', format='.2f'), alt.value(' '))
).transform_filter(
    nearest
)

interactive_co2 = alt.layer(
    co2_chart,
    vertical_line,
    interaction_dots,
    text_labels
).configure_axis(
    labelFontSize=12,
    labelFont='Roboto',
)
interactive_co2


alt.LayerChart(...)

In [21]:
ghg_df.head()

,EDGAR Country Code,Country,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,Aruba,0.045176,0.050063,0.060857,0.065847,0.065108,0.079128,0.078179,0.088896,...,0.463848,0.475081,0.476461,0.494114,0.467189,0.502077,0.575822,0.494731,0.479192,0.496683
1,AFG,Afghanistan,17.336192,17.292793,15.342251,15.966060,17.278443,17.984253,17.989107,18.153414,...,30.899954,31.214446,31.208495,30.567230,31.773007,31.439840,30.322851,28.150674,28.953697,29.117879
2,AGO,Angola,20.138364,20.018416,22.721001,24.375911,25.083208,23.889924,19.960419,24.620945,...,82.985493,85.186645,90.110604,87.096037,81.888045,78.215749,77.435411,71.682513,69.613898,66.480058
3,AIA,Anguilla,0.004256,0.004258,0.004358,0.004201,0.004451,0.004692,0.004526,0.004627,...,0.033171,0.033174,0.033318,0.033731,0.034494,0.033631,0.033000,0.028107,0.027267,0.028099
4,AIR,International Aviation,171.160455,171.147982,181.492019,189.261213,182.123246,176.107638,176.412012,192.354128,...,488.025465,503.952576,532.878284,555.785220,591.079314,617.430356,627.478302,299.325868,345.376001,425.963735


In [128]:
# print(ghg_df.Country.unique())
ghg_df_cleaned = ghg_df[ghg_df['Country'] != 'GLOBAL TOTAL']
ghg_df_cleaned = ghg_df_cleaned[ghg_df_cleaned['Country'] != 'EU27']
ghg_df_cleaned = ghg_df_cleaned[ghg_df_cleaned['Country'] != 'International Shipping']
ghg_df_cleaned = ghg_df_cleaned[ghg_df_cleaned['Country'] != 'International Aviation']
ghg_df_cleaned['Country'] = ghg_df_cleaned['Country'].replace('Italy, San Marino and the Holy See', 'Italy')
ghg_df_cleaned['Country'] = ghg_df_cleaned['Country'].replace('Spain and Andorra', 'Spain')
ghg_df_cleaned['Country'] = ghg_df_cleaned['Country'].replace('France and Monaco', 'France')

ghg_melted = ghg_df_cleaned.melt(id_vars=["EDGAR Country Code", "Country"],var_name="Year",value_name="Emissions")
# ghg_melted['Year'].dtypes
ghg_melted['Year'] = ghg_melted['Year'].astype(int)
# ghg_melted.Country.unique()

In [141]:
alt.data_transformers.disable_max_rows()
max_emissions = max(ghg_melted.Emissions)
year_slider = alt.binding_range(min=1970, max=2022, step=1)
slider_selection = alt.selection_point(bind=year_slider, fields=['Year'], name="Select", value=2022)

ghg_country_chart = alt.Chart(ghg_melted).mark_bar().add_params(
    slider_selection
).transform_filter(
    slider_selection
).transform_window(
    rank='rank(Emissions)',
    sort=[alt.SortField('Emissions', order='descending')],
    groupby=['Year']
).transform_filter(
    alt.datum.rank <= 15
).encode(
    y=alt.Y('Country:N', sort=alt.EncodingSortField(field='Emissions', order='descending'), axis=alt.Axis(title=None)),
    x=alt.X(
        'Emissions:Q',
        title='Mton CO2 equivalent',
        scale=alt.Scale(domain=(0, max_emissions))
    ),
    color=alt.Color('Emissions:Q', legend=alt.Legend(title="Emissions (Mton CO2 eq)"), scale=alt.Scale(domain=(0, max_emissions), scheme='reds')),
    tooltip=['Country', 'Year', 'Emissions']
)

ghg_country_chart

alt.Chart(...)

In [138]:
coastline_df.head()

,Country,The World Factbook[2],The World Factbook[2].1,World Resources Institute[1],World Resources Institute[1].1,Land area km2 (TWF)[3],Coast/area ratio (m/km2),Coast/area ratio (m/km2).1,Coast/area0.5 ratio
0,NaN,Rank,km,Rank,km,NaN,(TWF),(WRI),(TWF)
1,World[Note 2],—,"356,000",—,"1,634,701","148,940,000",2.39,11.0,29.2
2,Canada,1,"202,080",1,"265,523","9,093,507",22.2,29.2,67
3,Norway,2,"83,281[Note 3]",7,"53,199","304,282",274,175,151
4,Indonesia,3,"54,716",4,"95,181","1,811,569",30.2,52.5,40.7


In [166]:
coasts = pd.DataFrame(columns=['Country', 'Coastline Length'])
coasts['Country'] = coastline_df['Country']
coasts['Coastline Length'] = coastline_df['The World Factbook[2].1']
coasts = coasts[1:]
coasts = coasts.dropna(subset=['Country'])
coasts['Country'] = coasts['Country'].str.replace(r"\[.*?\]", "", regex=True)
coasts['Coastline Length'] = coasts['Coastline Length'].str.replace(r"\[.*?\]", "", regex=True)
coasts = coasts[coasts['Coastline Length'] != 'NA']
coasts = coasts.dropna(subset=['Coastline Length'])

# coasts['Coastline Length'] = coasts['Coastline Length'].astype(int)
coasts['Coastline Length'] = coasts['Coastline Length'].str.replace(',', '').astype(float)

coasts=coasts[coasts['Country'] != 'World']
# print(coasts['Country'].unique())
# set1 = set(coasts['Country'])
# set2 = set(ghg_df_cleaned['Country'])
# set1_unique = set1 - set2
# set2_unique = set2 - set1

# # Print the results
# print(f"Unique to Coasts: {set1_unique}")
# print(f"\nUnique to GHGs: {set2_unique}")
# print(disjoint_set)
coasts.head()


,Country,Coastline Length
2,Canada,202080.0
3,Norway,83281.0
4,Indonesia,54716.0
5,Greenland,44087.0
6,Russia,37653.0


In [146]:
coast_rankings = alt.Chart(coasts).mark_bar(   
).transform_window(
    rank = 'rank(Coastline Length)',
    sort = [alt.SortField('Coastline Length',order='descending')]
).transform_filter(
    alt.datum.rank <= 15
).encode(
    y = alt.Y('Country:N', sort=alt.EncodingSortField(field='Coastline Length', order='descending'), axis=alt.Axis(title=None)),
    x = alt.X('Coastline Length:Q', title='Coastline Length (km)')
)
coast_rankings | ghg_country_chart

alt.HConcatChart(...)

In [172]:
def name_to_iso_alpha3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return -1
        
coasts['iso_a3'] = coasts['Country'].apply(lambda x: name_to_iso_alpha3(x))
coasts = coasts[coasts['iso_a3'] != -1]
coasts.head()

,Country,Coastline Length,iso_a3
2,Canada,202080.0,CAN
3,Norway,83281.0,NOR
4,Indonesia,54716.0,IDN
5,Greenland,44087.0,GRL
7,Philippines,36289.0,PHL


In [176]:
import requests
import json

# Get the URL from vega_datasets
world_url = data.world_110m.url

# Download the TopoJSON data
response = requests.get(world_url)
topojson_data = response.json()

# Assuming the objects are called 'countries' as per your code
# This will print out the first few 'id' values in the 'countries' object
countries = topojson_data['objects']['countries']['geometries']
for country in countries[:10]:  # only show the first 10 to check
    print(country['id'])


4
24
8
784
32
51
10
260
36
40


In [173]:
from vega_datasets import data
world = data.world_110m.url
countries = alt.topo_feature(world, 'countries')

chloro = alt.Chart(countries).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    # lookup='id', from_=alt.LookupData(data=coasts, key='id', fields=['Country','Coastline Length'])
    lookup='id', from_=alt.LookupData(data=coasts, key='iso_a3', fields=['Country', 'Coastline Length'])
).encode(
    alt.Color('Coastline Length:Q', legend=alt.Legend()),
    # opacity=alt.condition(selection, alt.value(1), alt.value(0.25))
).project(
    type='mercator'
).properties(
    width=400,
    height=300
)

# base = alt.Chart(countries).mark_geoshape(
#     fill='lightgray',
#     stroke='white'
# ).properties(
#     width=800,
#     height=400
# )
# choropleth = alt.Chart(countries).mark_geoshape().encode(
#     color=alt.Color('properties.Coastline Length:Q', scale=alt.Scale(scheme='reds')),
#     tooltip=[alt.Tooltip('properties.Country:N'), alt.Tooltip('properties.Coastline Length:Q')]
# ).transform_lookup(
#     lookup='properties.name',
#     from_=alt.LookupData(data, 'Country', ['Coastline Length'])
# )

# # Composing the layered visualization
# map_chart = base + choropleth
chloro

alt.Chart(...)